<a href="https://colab.research.google.com/github/zero2me/dummy/blob/master/Copy_of_Internshala_Assignment_DNST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 7s 0us/step


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 12
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 12)   324         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 12)   48          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 12)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 163s 3ms/step - loss: 1.7229 - acc: 0.3484 - val_loss: 1.8999 - val_acc: 0.3670
Epoch 2/50
19200/50000 [==========>...................] - ETA: 1:28 - loss: 1.4507 - acc: 0.4567

50000/50000 [==============================] - 153s 3ms/step - loss: 1.3871 - acc: 0.4816 - val_loss: 1.3983 - val_acc: 0.4994
Epoch 3/50
45568/50000 [==========================>...] - ETA: 12s - loss: 1.2257 - acc: 0.5506

50000/50000 [==============================] - 153s 3ms/step - loss: 1.2215 - acc: 0.5530 - val_loss: 1.2788 - val_acc: 0.5453
Epoch 4/50
50000/50000 [==============================] - 154s 3ms/step - loss: 1.1110 - acc: 0.5934 - val_loss: 1.6285 - val_acc: 0.4999
Epoch 5/50
 3328/50000 [>.............................] - ETA: 2:15 - loss: 1.0370 - acc: 0.6169

50000/50000 [==============================] - 154s 3ms/step - loss: 1.0317 - acc: 0.6264 - val_loss: 1.1370 - val_acc: 0.6014
Epoch 6/50
39552/50000 [======================>.......] - ETA: 30s - loss: 0.9779 - acc: 0.6477

50000/50000 [==============================] - 154s 3ms/step - loss: 0.9768 - acc: 0.6488 - val_loss: 1.2246 - val_acc: 0.5842
Epoch 7/50
50000/50000 [==============================] - 154s 3ms/step - loss: 0.9342 - acc: 0.6611 - val_loss: 1.3511 - val_acc: 0.5887
Epoch 8/50
 1920/50000 [>.............................] - ETA: 2:18 - loss: 0.8872 - acc: 0.6875

...

50000/50000 [==============================] - 155s 3ms/step - loss: 0.5338 - acc: 0.8117 - val_loss: 0.7137 - val_acc: 0.7790
Epoch 45/50
38656/50000 [======================>.......] - ETA: 33s - loss: 0.5270 - acc: 0.8134

50000/50000 [==============================] - 155s 3ms/step - loss: 0.5290 - acc: 0.8134 - val_loss: 0.6665 - val_acc: 0.7916
Epoch 46/50
50000/50000 [==============================] - 155s 3ms/step - loss: 0.5249 - acc: 0.8137 - val_loss: 0.7279 - val_acc: 0.7763
Epoch 47/50
 1536/50000 [..............................] - ETA: 2:21 - loss: 0.5083 - acc: 0.8281

50000/50000 [==============================] - 155s 3ms/step - loss: 0.5247 - acc: 0.8164 - val_loss: 0.8955 - val_acc: 0.7364
Epoch 48/50
38656/50000 [======================>.......] - ETA: 33s - loss: 0.5159 - acc: 0.8174

50000/50000 [==============================] - 155s 3ms/step - loss: 0.5160 - acc: 0.8183 - val_loss: 0.6824 - val_acc: 0.7880
Epoch 49/50
50000/50000 [==============================] - 155s 3ms/step - loss: 0.5176 - acc: 0.8161 - val_loss: 0.6673 - val_acc: 0.7860
Epoch 50/50
 1536/50000 [..............................] - ETA: 2:21 - loss: 0.4596 - acc: 0.8477

50000/50000 [==============================] - 155s 3ms/step - loss: 0.5111 - acc: 0.8232 - val_loss: 0.7950 - val_acc: 0.7609


In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 13s 1ms/step
Test loss: 0.7949684534072876
Test accuracy: 0.7609


In [0]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')